In [44]:
# Install dependencies
import subprocess
import sys

try:
    subprocess.run(
        ["uv", "pip", "install", "-r", "requirements.txt"],
        capture_output=True,
        text=True,
        check=True
    )
    print("Dependencies installed successfully")
except (subprocess.CalledProcessError, FileNotFoundError):
    # Fallback to pip if uv is not available
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-r", "requirements.txt"],
        check=True
    )
    print("Dependencies installed successfully")

Dependencies installed successfully


In [45]:
# Import dependencies
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool
import json
import requests

In [46]:
# Environment variables
import os
from pathlib import Path

# Load from .env file if it exists
env_file = Path(".env")
if env_file.exists():
    with open(env_file) as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                os.environ[key] = value

# RunPod Pod endpoint (get from: https://www.runpod.io/console/pods)
# Deploy a pod with vLLM template and GPT-OSS 20B model
RUNPOD_ENDPOINT_URL = os.getenv("RUNPOD_ENDPOINT_URL", "YOUR_RUNPOD_ENDPOINT_HERE")
RUNPOD_API_KEY = os.getenv("RUNPOD_API_KEY", "YOUR_RUNPOD_API_KEY_HERE")

# Firecrawl API (get from: https://www.firecrawl.dev/)
FIRECRAWL_API_KEY = os.getenv("FIRECRAWL_API_KEY", "YOUR_FIRECRAWL_API_KEY_HERE")

print(f"Firecrawl API Key: {'Set' if FIRECRAWL_API_KEY != 'YOUR_FIRECRAWL_API_KEY_HERE' else 'Not set'}")
print(f"RunPod Endpoint: {'Set' if RUNPOD_ENDPOINT_URL != 'YOUR_RUNPOD_ENDPOINT_HERE' else 'Not set'}")

Firecrawl API Key: Set
RunPod Endpoint: Set


In [47]:
# Connect to Firecrawl MCP Server using LangChain MCP Adapters
from langchain_mcp_adapters.client import MultiServerMCPClient

# Configure Firecrawl MCP server
mcp_client = MultiServerMCPClient({
    "firecrawl": {
        "transport": "stdio",
        "command": "npx",
        "args": ["-y", "firecrawl-mcp"],
        "env": {
            **os.environ,
            "FIRECRAWL_API_KEY": FIRECRAWL_API_KEY
        }
    }
})

# Get tools from MCP server
# Note: Jupyter notebooks have a running event loop, so we use await directly
firecrawl_tools = await mcp_client.get_tools()

print(f"Connected to Firecrawl MCP server")
print(f"Available tools: {[tool.name for tool in firecrawl_tools]}")

Connected to Firecrawl MCP server
Available tools: ['firecrawl_scrape', 'firecrawl_map', 'firecrawl_search', 'firecrawl_crawl', 'firecrawl_check_crawl_status', 'firecrawl_extract']


# AI Job Application Assistant Workshop

## Overview

In this workshop, you'll build an intelligent agent that helps students find jobs, evaluate their fit, and write personalized cover letters.

### What You'll Learn
- How LLM agents make decisions
- vLLM for fast, local inference
- LangChain v1 agent architecture
- Multi-step reasoning with tools
- Real-time web scraping with MCP
- Production-ready integrations

### The Problem
Students manually apply to dozens of jobs:
- Copy-pasting job descriptions
- Rewriting cover letters for each role
- Uncertain if they're qualified
- No systematic approach

### Our Solution
An agent that autonomously:
1. **Finds real jobs** - Scrapes live job boards using Firecrawl MCP
2. **Scores resume fit** - Analyzes skills match
3. **Generates personalized cover letters** - Tailored to each role
4. **Makes intelligent tool selections** - ReAct-style reasoning

### Architecture
```
User Query → vLLM Agent → Tool Selection → Response
                   ↓
        [Job Search (Firecrawl MCP) | Resume Scorer | Cover Letter]
```

### Key Technologies
- **LangChain v1** - Modern agent framework with `create_agent()`
- **vLLM** - Fast LLM inference on GPUs
- **Hermes-2-Pro-Mistral-7B** - Excellent tool calling support
- **Firecrawl MCP** - Production web scraping via Model Context Protocol
- **RunPod** - GPU infrastructure for vLLM

**This is a realistic end-to-end demo with no mock data!**

## Step 1: Setup vLLM

We'll use vLLM running on a RunPod Pod with PyTorch 2.4.0 template.

**Model Used**: Hermes-2-Pro-Mistral-7B (excellent tool calling support)

**Setup Steps**:
1. Deploy RunPod pod with PyTorch 2.4.0 template (50GB+ container disk)
2. SSH into pod and run:
   ```bash
   pip install vllm requests
   python -m vllm.entrypoints.openai.api_server \
       --model NousResearch/Hermes-2-Pro-Mistral-7B \
       --host 0.0.0.0 \
       --port 8000 \
       --trust-remote-code \
       --enable-auto-tool-choice \
       --tool-call-parser hermes
   ```
3. Copy endpoint URL (https://[pod-id]-8000.proxy.runpod.net) to .env file

In [41]:
# Initialize vLLM on RunPod Pod
from langchain_openai import ChatOpenAI
from pydantic import SecretStr

if RUNPOD_ENDPOINT_URL == "YOUR_RUNPOD_ENDPOINT_HERE":
    raise ValueError(
        "RunPod endpoint not configured. Please:\n"
        "1. Deploy a RunPod pod with PyTorch template\n"
        "2. Install vLLM and start server with Hermes-2-Pro-Mistral-7B\n"
        "3. Add RUNPOD_ENDPOINT_URL and RUNPOD_API_KEY to .env file"
    )

# LangChain v1 / Modern OpenAI SDK parameters
llm = ChatOpenAI(
    api_key=SecretStr(RUNPOD_API_KEY),
    base_url=RUNPOD_ENDPOINT_URL,
    model="NousResearch/Hermes-2-Pro-Mistral-7B",
    max_completion_tokens=512,
    temperature=0.7
)

print("Connected to RunPod pod")

# Test the connection
try:
    response = llm.invoke("Say 'Hello from vLLM!' in one sentence.")
    print(f"Test response: {response.content}")
except Exception as e:
    print(f"Connection test failed: {e}")
    print("The model may still be loading. Wait a few minutes and try again.")

Connected to RunPod pod
Test response: "Hello from vLLM! Wishing you warm greetings and a fantastic day ahead!"


## Step 2: Connect to MCP Tools

We've connected to Firecrawl MCP server to get web scraping tools. The agent will use these to fetch job listings from real websites.

**Key Point:** The agent receives tools directly and decides when/how to use them based on the system prompt.

## Step 3: Build the Agent with LangChain v1

Now we create the agent using **LangChain v1's modern architecture**. The agent will:
1. Receive Firecrawl MCP tools directly (no wrappers!)
2. Use its reasoning to decide when to scrape job sites
3. Construct appropriate URLs based on system prompt guidance
4. Parse and analyze scraped content
5. Score resumes and write cover letters using its native language abilities

**LangChain v1 Architecture:**
- Uses `create_agent()` - simple, clean API
- Built on LangGraph for durable execution and streaming
- Message-based invocation: `agent.invoke({"messages": [...]})`
- System prompt guides tool usage

**Key Insight:**
The LLM agent is intelligent enough to:
- Construct job site URLs (e.g., naukri.com/python-developer-jobs-in-bangalore)
- Use firecrawl_scrape appropriately
- Parse markdown results
- Compare skills and write cover letters without separate tools

This demonstrates **true agentic AI** - the agent reasons about how to use tools, not just executing predefined wrappers.

In [56]:
# Create the agent using LangChain v1

# System prompt guides the agent on how to use tools
system_prompt = """You are an intelligent job search assistant helping students find and apply to jobs.

You have access to Firecrawl MCP tools for web scraping:
- firecrawl_scrape: Scrape a single URL and extract content as markdown
- firecrawl_search: Search for jobs using the keywords

Common job sites and URL patterns:
- Naukri.com (India): https://www.naukri.com/{role-with-dashes}-jobs-in-{location}
  Example: https://www.naukri.com/python-developer-jobs-in-bangalore
- LinkedIn: https://www.linkedin.com/jobs/search/?keywords={role}&location={location}

When asked to find jobs:
1. Construct the appropriate job site URL
2. Use firecrawl_scrape to get job listings
3. Analyze the scraped content and present top matches

You can also help with:
- Resume scoring: Compare candidate skills with job requirements using your reasoning
- Cover letters: Generate personalized cover letters using your language abilities

Be helpful, encouraging, and provide actionable advice."""

# Create agent - pass Firecrawl tools directly
agent = create_agent(
    model=llm,
    tools=firecrawl_tools,
    system_prompt=system_prompt
)

print("Agent created successfully!")
print(f"Tools available: {len(firecrawl_tools)}")

# Quick test (MCP tools require async)
test_response = await agent.ainvoke({
    "messages": [{"role": "user", "content": "What can you help me with?"}]
})

print("\nAgent test response:")
print(test_response["messages"][-1].content)

Agent created successfully!
Tools available: 6

Agent test response:
I can assist you with the following tasks:

1. Job searching: Using Firecrawl tools, I can scrape job listings from websites like Naukri.com (for Indian jobs) and LinkedIn. You can provide the role and location you're interested in, and I'll find relevant job openings.

2. Resume scoring: If you have a list of skills for a particular job or role, I can compare them with the job requirements and provide a score on how well the candidate's skills match the job requirements.

3. Cover letters: If you need help in crafting a cover letter, I can generate a personalized cover letter using my language abilities, based on the job description and your experience.

Please let me know which of these tasks you'd like me to help with, and provide any necessary details.


## Live Demos

Now let's see the agent in action with progressively complex queries.

### Demo 1: Simple Job Search

Query: "Find Python developer jobs in Bangalore"

**Expected Agent Behavior:**
1. Recognizes need to scrape job site
2. Constructs URL: `https://www.naukri.com/python-developer-jobs-in-bangalore`
3. Uses `firecrawl_scrape` tool
4. Analyzes and presents results

**This will scrape real, live job data from Naukri.com using Firecrawl MCP!**

In [57]:
# Demo 1: Simple job search
response = await agent.ainvoke({
    "messages": [{"role": "user", "content": "Find Python developer jobs in Bangalore"}]
})

print("\nAgent Response:")
print(response["messages"][-1].content)


Agent Response:
Here are the top 5 Python developer job listings in Bangalore:

1. [57,564 Python Job Vacancies In Bangalore - Bengaluru - Naukri.com](https://www.naukri.com/python-jobs-in-bangalore)
    - Python jobs in Bangalore ; Python Software Developer. HCLTech · Hybrid - Hyderabad, Bengaluru ; Python Software Developer - ENG @Infosys - PAN INDIA.

2. [Python Developer jobs in Bengaluru, Karnataka - Indeed](https://in.indeed.com/q-python-developer-l-bengaluru,-karnataka-jobs.html)
    - 3282 Python Developer jobs available in Bengaluru, Karnataka on Indeed.com.

3. [2,000+ Python Developer jobs in Bengaluru, Karnataka, India (196 ...](https://in.linkedin.com/jobs/python-developer-jobs-bengaluru)
    - Get notified about new Python Developer jobs in Bengaluru, Karnataka, India. Sign in to create job alert. 2,000+ Python Developer Jobs in Bengaluru, Karnataka, ...

4. [Python Developer - Caterpillar Careers](https://careers.caterpillar.com/en/jobs/r0000317195/python-developer/)
  

### Demo 2: Multi-Step Reasoning with Resume Scoring

Query: "Find Python jobs in Bangalore. My skills are Python, FastAPI, and Docker. Which job should I apply to?"

**Expected Agent Behavior:**
1. Constructs Naukri.com URL
2. Uses `firecrawl_scrape` to get job listings
3. **Uses its own reasoning** to compare candidate skills with job requirements
4. Recommends best match with justification

**Key Insight:** No separate `score_resume` tool needed! The LLM agent is smart enough to compare skills and provide recommendations using its native language understanding.

**This demonstrates autonomous multi-step reasoning with real scraped data.**

In [58]:
# Demo 2: Multi-step reasoning
response = await agent.ainvoke({
    "messages": [{"role": "user", "content": """Find Python developer jobs in Bangalore. 
    My skills are: Python, FastAPI, Docker. 
    Which job should I apply to and why?"""}]
})

print("\nAgent Response:")
print(response["messages"][-1].content)

McpError: MCP error -32602: Tool 'firecrawl_search' parameter validation failed: sources.0: Invalid input: expected object, received string. Please check the parameter types and values according to the tool's schema.

### Demo 3: Full Autonomous Workflow

Query: "Find the best Python job in Hyderabad for someone with Python and AWS skills, then write a cover letter for it."

**Expected Agent Behavior:**
1. Constructs URL and scrapes Hyderabad job listings
2. **Uses its reasoning** to analyze which job best matches Python + AWS skills
3. **Uses its language abilities** to generate a personalized cover letter for that specific company

**Key Insights:**
- No separate tools for resume scoring or cover letter generation needed!
- The LLM agent handles all reasoning tasks naturally
- Only uses external tools (Firecrawl) when it needs to fetch external data
- Everything else is handled by the agent's native intelligence

**This demonstrates full autonomous behavior from a single prompt - the agent plans and executes a complete workflow with real data!**

In [ ]:
# Demo 3: Full autonomous workflow
response = await agent.ainvoke({
    "messages": [{"role": "user", "content": """Find the best Python developer job in Hyderabad for someone 
    with Python and AWS skills, then write a personalized cover letter for it."""}]
})

print("\nAgent Response:")
print(response["messages"][-1].content)

## Summary

You've built a production-ready LangChain v1 agent with proper MCP integration!

### What You Built
- vLLM-powered agent with LangChain v1
- Direct MCP tool integration (Firecrawl)
- Autonomous decision-making with multi-step reasoning
- Clean, maintainable architecture

### Architecture

```python
# Get MCP tools
firecrawl_tools = await mcp_client.get_tools()

# Pass directly to agent
agent = create_agent(
    model=llm,
    tools=firecrawl_tools,
    system_prompt="..."
)
```

### Key Principles

1. **Trust the LLM Agent** - It's smart enough to construct URLs, parse results, and reason about data
2. **Tools for External Data** - Use tools to fetch/scrape data from the web or databases
3. **Native Reasoning** - Let the LLM handle analysis, comparison, and generation
4. **System Prompt as Guide** - Teach the agent patterns through clear instructions

### LangChain v1 Features
- `create_agent()` - Modern agent creation
- Built on LangGraph for streaming and persistence
- Message-based API: `agent.invoke({"messages": [...]})`
- Direct MCP tool integration via `langchain-mcp-adapters`

### Extension Ideas

**Quick:**
- Add more job sites via system prompt URL patterns
- Implement streaming with `agent.stream()`
- Add conversation memory

**Intermediate:**
- Parse PDF resumes with LangChain document loaders
- Build Streamlit UI with real-time streaming
- Integrate email APIs for automated applications

**Advanced:**
- Multi-agent system with specialized roles
- Human-in-the-loop workflow with LangGraph
- Application tracking database
- Browser automation for authenticated sites

### Resources

**LangChain v1:**
- [Documentation](https://python.langchain.com/)
- [Build an Agent Tutorial](https://python.langchain.com/docs/tutorials/agents/)
- [Migration Guide](https://docs.langchain.com/oss/python/migrate/langchain-v1)
- [LangGraph](https://langchain-ai.github.io/langgraph/)

**MCP:**
- [Model Context Protocol](https://modelcontextprotocol.io/)
- [LangChain MCP Adapters](https://docs.langchain.com/oss/python/langchain/mcp)
- [Firecrawl MCP Server](https://github.com/mendableai/firecrawl-mcp)

**vLLM:**
- [Documentation](https://docs.vllm.ai/)
- [Hermes Models](https://huggingface.co/NousResearch)

### Production Tips

**Streaming:**
```python
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "Find jobs"}]},
    stream_mode="updates"
):
    print(chunk)
```

**Persistence:**
```python
from langgraph.checkpoint.memory import MemorySaver

agent = create_agent(
    model=llm,
    tools=firecrawl_tools,
    checkpointer=MemorySaver()
)
```

**Error Handling:**
```python
try:
    response = agent.invoke({"messages": [...]})
except Exception as e:
    print(f"Agent error: {e}")
    # Implement retry logic
```